# Technical Indicators

This notebook computes and analyzes a variety``` financial technical indicators commonly used for stock```ice and trend prediction in```e StockPredictionPro platform```**Technical Indicators Covered:**
- Moving Averages (SMA, EMA)
- Relative Strength Index (RSI)
- Moving Average Convergence Divergence (MACD)
- Bollinger Bands
- Stochastic Oscillator
- Average True Range (ATR)

**Objectives:**
- Calculate essential technical indicators from```LCV data
- Create features for machine learning model```aining
- Provide signal analysis for trading```rategies
- Export engineered features for model development```hese features will be integrated into our prediction```dels and trading```gorithms.


In [2]:
# ============================================
# Technical Indicators - Library Imports and Setup
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
sns.set_theme(style='darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
%matplotlib inline

# Try importing TA-Lib for advanced technical analysis
try:
    import talib as ta
    TALIB_AVAILABLE = True
    print("✅ TA-Lib imported successfully - using optimized indicators")
except ImportError:
    TALIB_AVAILABLE = False
    print("📝 TA-Lib not available - using pandas-based calculations")
    print("   Install with: pip install TA-Lib")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("📊 Ready for technical indicator calculation")


✅ TA-Lib imported successfully - using optimized indicators
📊 Ready for technical indicator calculation


In [3]:
# ============================================
# Load Market Data for Technical Analysis
# ============================================

# Load historical OHLCV data
data_path = '../data/raw/market_data.csv'

try:
    # Load real market data
    df = pd.read_csv(data_path, parse_dates=['date'], index_col='date')
    print(f"✅ Market data loaded from {data_path}")
    print(f"📊 Data shape: {df.shape}")
    
except FileNotFoundError:
    print("📝 Market data file not found. Generating realistic sample data...")
    
    # Generate realistic OHLCV data for demonstration
    np.random.seed(42)
    n_periods = 1000
    dates = pd.date_range(start='2020-01-01', periods=n_periods, freq='D')
    
    # Generate realistic price movements
    initial_price = 100
    daily_returns = np.random.normal(0.0005, 0.02, n_periods)  # ~0.05% daily return, 2% volatility
    
    # Create price series with realistic movements
    prices = [initial_price]
    for ret in daily_returns[1:]:
        prices.append(prices[-1] * (1 + ret))
    
    # Create OHLCV DataFrame
    df = pd.DataFrame(index=dates)
    df['close'] = prices
    df['open'] = df['close'].shift(1) * (1 + np.random.normal(0, 0.005, n_periods))
    df['high'] = np.maximum(df['open'], df['close']) * (1 + np.abs(np.random.normal(0, 0.01, n_periods)))
    df['low'] = np.minimum(df['open'], df['close']) * (1 - np.abs(np.random.normal(0, 0.01, n_periods)))
    df['volume'] = np.random.lognormal(15, 0.5, n_periods).astype(int)
    
    # Ensure OHLC relationships are correct
    df['high'] = np.maximum(df['high'], df[['open', 'close']].max(axis=1))
    df['low'] = np.minimum(df['low'], df[['open', 'close']].min(axis=1))
    
    # Remove NaN values
    df = df.dropna()
    
    print("✅ Realistic sample data generated")

# Display basic information
print(f"\n📈 Dataset Overview:")
print(f"   • Date Range: {df.index.min().date()} to {df.index.max().date()}")
print(f"   • Total Records: {len(df):,}")
print(f"   • Columns: {list(df.columns)}")

print(f"\n📋 Data Preview:")
display(df.head())


📝 Market data file not found. Generating realistic sample data...
✅ Realistic sample data generated

📈 Dataset Overview:
   • Date Range: 2020-01-02 to 2022-09-26
   • Total Records: 999
   • Columns: ['close', 'open', 'high', 'low', 'volume']

📋 Data Preview:


,close,open,high,low,volume
2020-01-02,99.773471,100.462317,100.607504,98.915035,3218410
2020-01-03,101.115801,99.803219,101.917063,99.390427,3298599
2020-01-04,104.246406,100.788723,104.567445,98.886147,4140450
2020-01-05,103.810337,104.610343,106.591260,103.232577,1650470
2020-01-06,103.376125,104.014576,104.236432,101.995556,4396343


In [4]:
# ============================================
# Moving Averages Calculation
# ============================================

print("=" * 60)
print("MOVING AVERAGES CALCULATION")
print("=" * 60)

# Simple Moving Averages (SMA)
sma_periods = [5, 10, 20, 50, 200]
for period in sma_periods:
    df[f'SMA_{period}'] = df['close'].rolling(window=period).mean()
    print(f"✅ SMA_{period} calculated")

# Exponential Moving Averages (EMA)
ema_periods = [12, 26, 50]
for period in ema_periods:
    df[f'EMA_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    print(f"✅ EMA_{period} calculated")

# Volume Weighted Average Price (VWAP)
df['VWAP'] = (df['close'] * df['volume']).rolling(window=20).sum() / df['volume'].rolling(window=20).sum()
print("✅ VWAP calculated")

# Moving Average Ratios (for trend strength)
df['SMA_ratio_5_20'] = df['SMA_5'] / df['SMA_20']
df['SMA_ratio_20_50'] = df['SMA_20'] / df['SMA_50']
print("✅ Moving average ratios calculated")

print(f"\n📊 Moving Averages Preview:")
ma_columns = [col for col in df.columns if 'MA' in col or 'VWAP' in col]
display(df[ma_columns].head(10))

print(f"\n📈 Moving Averages Summary:")
print(f"   • Simple Moving Averages: {len([c for c in ma_columns if 'SMA' in c])}")
print(f"   • Exponential Moving Averages: {len([c for c in ma_columns if 'EMA' in c])}")
print(f"   • Additional indicators: VWAP, ratios")


MOVING AVERAGES CALCULATION
✅ SMA_5 calculated
✅ SMA_10 calculated
✅ SMA_20 calculated
✅ SMA_50 calculated
✅ SMA_200 calculated
✅ EMA_12 calculated
✅ EMA_26 calculated
✅ EMA_50 calculated
✅ VWAP calculated
✅ Moving average ratios calculated

📊 Moving Averages Preview:


,SMA_5,SMA_10,SMA_20,SMA_50,SMA_200,EMA_12,EMA_26,EMA_50,VWAP,SMA_ratio_5_20,SMA_ratio_20_50
2020-01-02,NaN,NaN,NaN,NaN,NaN,99.773471,99.773471,99.773471,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,99.979984,99.872903,99.826112,NaN,NaN,NaN
2020-01-04,NaN,NaN,NaN,NaN,NaN,100.636356,100.196866,99.999457,NaN,NaN,NaN
2020-01-05,NaN,NaN,NaN,NaN,NaN,101.124661,100.464531,100.148903,NaN,NaN,NaN
2020-01-06,102.464428,NaN,NaN,NaN,NaN,101.471040,100.680205,100.275461,NaN,NaN,NaN
2020-01-07,103.848308,NaN,NaN,NaN,NaN,102.274399,101.125587,100.527124,NaN,NaN,NaN
2020-01-08,105.301911,NaN,NaN,NaN,NaN,103.214309,101.663234,100.835229,NaN,NaN,NaN
2020-01-09,105.936697,NaN,NaN,NaN,NaN,103.861390,102.089686,101.093469,NaN,NaN,NaN
2020-01-10,106.902567,NaN,NaN,NaN,NaN,104.596513,102.574871,101.389399,NaN,NaN,NaN
2020-01-11,107.764761,105.114595,NaN,NaN,NaN,105.071987,102.953554,101.636367,NaN,NaN,NaN



📈 Moving Averages Summary:
   • Simple Moving Averages: 7
   • Exponential Moving Averages: 3
   • Additional indicators: VWAP, ratios


In [5]:
# ============================================
# Relative Strength Index (RSI) Calculation
# ============================================

print("=" * 60)
print("RSI (RELATIVE STRENGTH INDEX) CALCULATION")
print("=" * 60)

def calculate_rsi(prices, period=14):
    """
    Calculate Relative Strength Index (RSI)
    RSI = 100 - (100 / (1 + RS))
    RS = Average Gain / Average Loss
    """
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and loss using Wilder's smoothing
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
    # For subsequent values, use Wilder's smoothing
    for i in range(period, len(avg_gain)):
        avg_gain.iloc[i] = (avg_gain.iloc[i-1] * (period - 1) + gain.iloc[i]) / period
        avg_loss.iloc[i] = (avg_loss.iloc[i-1] * (period - 1) + loss.iloc[i]) / period
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RSI for different periods
rsi_periods = [14, 21]
for period in rsi_periods:
    df[f'RSI_{period}'] = calculate_rsi(df['close'], period)
    print(f"✅ RSI_{period} calculated")

# RSI-based signals
df['RSI_oversold'] = (df['RSI_14'] < 30).astype(int)  # Oversold signal
df['RSI_overbought'] = (df['RSI_14'] > 70).astype(int)  # Overbought signal

print(f"\n📊 RSI Statistics:")
rsi_stats = df['RSI_14'].describe()
print(rsi_stats)

print(f"\n🔍 RSI Signal Summary:")
oversold_count = df['RSI_oversold'].sum()
overbought_count = df['RSI_overbought'].sum()
print(f"   • Oversold signals (RSI < 30): {oversold_count}")
print(f"   • Overbought signals (RSI > 70): {overbought_count}")
print(f"   • Neutral periods: {len(df) - oversold_count - overbought_count}")

print(f"\n📋 RSI Preview:")
rsi_columns = [col for col in df.columns if 'RSI' in col]
display(df[rsi_columns].tail(10))


RSI (RELATIVE STRENGTH INDEX) CALCULATION
✅ RSI_14 calculated
✅ RSI_21 calculated

📊 RSI Statistics:
count    986.000000
mean      52.321551
std       12.094987
min       22.219003
25%       43.909044
50%       51.170726
75%       59.294390
max       88.141314
Name: RSI_14, dtype: float64

🔍 RSI Signal Summary:
   • Oversold signals (RSI < 30): 19
   • Overbought signals (RSI > 70): 93
   • Neutral periods: 887

📋 RSI Preview:


,RSI_14,RSI_21,RSI_oversold,RSI_overbought
2022-09-17,48.316794,49.061394,0,0
2022-09-18,39.754630,43.234760,0,0
2022-09-19,38.968393,42.672182,0,0
2022-09-20,36.669213,41.022201,0,0
2022-09-21,33.542638,38.716285,0,0
2022-09-22,32.769009,38.137544,0,0
2022-09-23,42.824476,44.324996,0,0
2022-09-24,46.110931,46.454162,0,0
2022-09-25,43.854486,44.953742,0,0
2022-09-26,46.883676,46.902442,0,0


In [6]:
# ============================================
# MACD (Moving Average Convergence Divergence) Calculation
# ============================================

print("=" * 60)
print("MACD CALCULATION")
print("=" * 60)

def calculate_macd(prices, fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD (Moving Average Convergence Divergence)
    MACD Line = EMA(fast) - EMA(slow)
    Signal Line = EMA(MACD Line)
    Histogram = MACD Line - Signal Line
    """
    ema_fast = prices.ewm(span=fast_period).mean()
    ema_slow = prices.ewm(span=slow_period).mean()
    
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal_period).mean()
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram

# Calculate MACD components
df['MACD'], df['MACD_signal'], df['MACD_histogram'] = calculate_macd(df['close'])

# MACD-based trading signals
df['MACD_bullish_crossover'] = ((df['MACD'] > df['MACD_signal']) & 
                                (df['MACD'].shift(1) <= df['MACD_signal'].shift(1))).astype(int)
df['MACD_bearish_crossover'] = ((df['MACD'] < df['MACD_signal']) & 
                                (df['MACD'].shift(1) >= df['MACD_signal'].shift(1))).astype(int)

print("✅ MACD components calculated")
print("✅ MACD trading signals generated")

print(f"\n📊 MACD Statistics:")
macd_stats = df[['MACD', 'MACD_signal', 'MACD_histogram']].describe()
display(macd_stats)

print(f"\n🔍 MACD Signal Summary:")
bullish_signals = df['MACD_bullish_crossover'].sum()
bearish_signals = df['MACD_bearish_crossover'].sum()
print(f"   • Bullish crossovers: {bullish_signals}")
print(f"   • Bearish crossovers: {bearish_signals}")

print(f"\n📋 MACD Preview:")
macd_columns = [col for col in df.columns if 'MACD' in col]
display(df[macd_columns].tail(10))


MACD CALCULATION
✅ MACD components calculated
✅ MACD trading signals generated

📊 MACD Statistics:


,MACD,MACD_signal,MACD_histogram
count,999.000000,999.000000,999.000000
mean,0.721968,0.737105,-0.015137
std,2.975204,2.807400,0.845507
min,-5.563273,-5.044221,-3.994602
25%,-1.150289,-0.965357,-0.465189
50%,0.320509,0.313653,-0.024270
75%,1.958592,2.026917,0.464809
max,11.714483,10.797574,2.763446



🔍 MACD Signal Summary:
   • Bullish crossovers: 39
   • Bearish crossovers: 38

📋 MACD Preview:


,MACD,MACD_signal,MACD_histogram,MACD_bullish_crossover,MACD_bearish_crossover
2022-09-17,-2.454746,-2.413447,-0.041299,0,0
2022-09-18,-2.834611,-2.497680,-0.336931,0,0
2022-09-19,-3.169709,-2.632086,-0.537623,0,0
2022-09-20,-3.601576,-2.825984,-0.775592,0,0
2022-09-21,-4.200278,-3.100843,-1.099435,0,0
2022-09-22,-4.698969,-3.420468,-1.278501,0,0
2022-09-23,-4.487457,-3.633866,-0.853591,0,0
2022-09-24,-4.062860,-3.719665,-0.343196,0,0
2022-09-25,-3.856576,-3.747047,-0.109529,0,0
2022-09-26,-3.464143,-3.690466,0.226323,1,0


In [7]:
# ============================================
# Bollinger Bands Calculation
# ============================================

print("=" * 60)
print("BOLLINGER BANDS CALCULATION")
print("=" * 60)

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    """
    Calculate Bollinger Bands
    Middle Band = SMA(period)
    Upper Band = Middle Band + (std_dev * standard deviation)
    Lower Band = Middle Band - (std_dev * standard deviation)
    """
    middle_band = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    
    upper_band = middle_band + (std_dev * std)
    lower_band = middle_band - (std_dev * std)
    
    return upper_band, middle_band, lower_band

# Calculate Bollinger Bands
df['BB_upper'], df['BB_middle'], df['BB_lower'] = calculate_bollinger_bands(df['close'])

# Bollinger Band indicators
df['BB_width'] = df['BB_upper'] - df['BB_lower']  # Band width (volatility measure)
df['BB_position'] = (df['close'] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])  # Position within bands
df['BB_squeeze'] = (df['BB_width'] < df['BB_width'].rolling(window=20).mean()).astype(int)  # Squeeze detection

# Bollinger Band signals
df['BB_breakout_upper'] = (df['close'] > df['BB_upper']).astype(int)
df['BB_breakout_lower'] = (df['close'] < df['BB_lower']).astype(int)

print("✅ Bollinger Bands calculated")
print("✅ Bollinger Band indicators and signals generated")

print(f"\n📊 Bollinger Bands Statistics:")
bb_stats = df[['BB_upper', 'BB_middle', 'BB_lower', 'BB_width', 'BB_position']].describe()
display(bb_stats)

print(f"\n🔍 Bollinger Band Signal Summary:")
upper_breakouts = df['BB_breakout_upper'].sum()
lower_breakouts = df['BB_breakout_lower'].sum()
squeeze_periods = df['BB_squeeze'].sum()
print(f"   • Upper band breakouts: {upper_breakouts}")
print(f"   • Lower band breakouts: {lower_breakouts}")
print(f"   • Squeeze periods: {squeeze_periods}")

print(f"\n📋 Bollinger Bands Preview:")
bb_columns = [col for col in df.columns if 'BB_' in col]
display(df[bb_columns].tail(10))


BOLLINGER BANDS CALCULATION
✅ Bollinger Bands calculated
✅ Bollinger Band indicators and signals generated

📊 Bollinger Bands Statistics:


,BB_upper,BB_middle,BB_lower,BB_width,BB_position
count,980.000000,980.000000,980.000000,980.000000,980.000000
mean,126.126753,117.940523,109.754293,16.372460,0.536539
std,35.633237,32.148991,29.076887,9.805883,0.323472
min,84.299792,81.338091,77.007847,4.555983,-0.213144
25%,105.862240,99.602291,92.843303,9.963767,0.271231
50%,115.205728,108.884827,101.918864,13.526519,0.523932
75%,133.665497,125.988701,117.588843,20.341242,0.792849
max,240.256609,219.354824,202.809468,56.812129,1.380956



🔍 Bollinger Band Signal Summary:
   • Upper band breakouts: 79
   • Lower band breakouts: 31
   • Squeeze periods: 518

📋 Bollinger Bands Preview:


,BB_upper,BB_middle,BB_lower,BB_width,BB_position,BB_squeeze,BB_breakout_upper,BB_breakout_lower
2022-09-17,226.543796,206.790374,187.036952,39.506844,0.458559,1,0,0
2022-09-18,224.176318,205.489765,186.803213,37.373104,0.269583,1,0,0
2022-09-19,220.170649,203.952471,187.734293,32.436356,0.254938,1,0,0
2022-09-20,216.313090,202.428719,188.544347,27.768743,0.175873,1,0,0
2022-09-21,212.444214,200.810365,189.176516,23.267698,0.020349,1,0,0
2022-09-22,210.717068,199.557401,188.397735,22.319332,0.012584,1,0,0
2022-09-23,208.381370,198.739672,189.097973,19.283397,0.334934,1,0,0
2022-09-24,208.099746,198.560501,189.021257,19.078489,0.479052,1,0,0
2022-09-25,208.092708,198.587599,189.082490,19.010218,0.363685,1,0,0
2022-09-26,208.084041,198.578309,189.072577,19.011464,0.487397,1,0,0


In [8]:
# ============================================
# Stochastic Oscillator Calculation
# ============================================

print("=" * 60)
print("STOCHASTIC OSCILLATOR CALCULATION")
print("=" * 60)

def calculate_stochastic(high, low, close, k_period=14, d_period=3):
    """
    Calculate Stochastic Oscillator
    %K = 100 * (Current Close - Lowest Low) / (Highest High - Lowest Low)
    %D = SMA of %K over d_period
    """
    lowest_low = low.rolling(window=k_period).min()
    highest_high = high.rolling(window=k_period).max()
    
    k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    d_percent = k_percent.rolling(window=d_period).mean()
    
    return k_percent, d_percent

# Calculate Stochastic Oscillator
df['Stoch_K'], df['Stoch_D'] = calculate_stochastic(df['high'], df['low'], df['close'])

# Stochastic signals
df['Stoch_oversold'] = (df['Stoch_K'] < 20).astype(int)  # Oversold
df['Stoch_overbought'] = (df['Stoch_K'] > 80).astype(int)  # Overbought
df['Stoch_bullish_crossover'] = ((df['Stoch_K'] > df['Stoch_D']) & 
                                 (df['Stoch_K'].shift(1) <= df['Stoch_D'].shift(1))).astype(int)
df['Stoch_bearish_crossover'] = ((df['Stoch_K'] < df['Stoch_D']) & 
                                 (df['Stoch_K'].shift(1) >= df['Stoch_D'].shift(1))).astype(int)

print("✅ Stochastic Oscillator calculated")
print("✅ Stochastic signals generated")

print(f"\n📊 Stochastic Statistics:")
stoch_stats = df[['Stoch_K', 'Stoch_D']].describe()
display(stoch_stats)

print(f"\n🔍 Stochastic Signal Summary:")
stoch_oversold = df['Stoch_oversold'].sum()
stoch_overbought = df['Stoch_overbought'].sum()
stoch_bullish = df['Stoch_bullish_crossover'].sum()
stoch_bearish = df['Stoch_bearish_crossover'].sum()

print(f"   • Oversold periods (%K < 20): {stoch_oversold}")
print(f"   • Overbought periods (%K > 80): {stoch_overbought}")
print(f"   • Bullish crossovers: {stoch_bullish}")
print(f"   • Bearish crossovers: {stoch_bearish}")

print(f"\n📋 Stochastic Preview:")
stoch_columns = [col for col in df.columns if 'Stoch' in col]
display(df[stoch_columns].tail(10))


STOCHASTIC OSCILLATOR CALCULATION
✅ Stochastic Oscillator calculated
✅ Stochastic signals generated

📊 Stochastic Statistics:


,Stoch_K,Stoch_D
count,986.000000,984.000000
mean,50.511152,50.553558
std,29.384726,27.436419
min,0.173169,0.549826
25%,25.040160,25.900211
50%,49.671531,49.734272
75%,77.697102,76.012674
max,99.994101,97.585897



🔍 Stochastic Signal Summary:
   • Oversold periods (%K < 20): 197
   • Overbought periods (%K > 80): 225
   • Bullish crossovers: 215
   • Bearish crossovers: 215

📋 Stochastic Preview:


,Stoch_K,Stoch_D,Stoch_oversold,Stoch_overbought,Stoch_bullish_crossover,Stoch_bearish_crossover
2022-09-17,59.452981,50.208273,0,0,0,0
2022-09-18,31.534986,46.754009,0,0,0,1
2022-09-19,26.735900,39.241289,0,0,0,0
2022-09-20,12.607936,23.626274,1,0,0,0
2022-09-21,16.577144,18.640326,1,0,0,0
2022-09-22,12.465530,13.883537,1,0,0,0
2022-09-23,43.950385,24.331020,0,0,1,0
2022-09-24,54.525676,36.980531,0,0,0,0
2022-09-25,45.735434,48.070499,0,0,0,1
2022-09-26,55.247915,51.836342,0,0,1,0


In [9]:
# ============================================
# ATR and Volatility Indicators Calculation
# ============================================

print("=" * 60)
print("VOLATILITY INDICATORS CALCULATION")
print("=" * 60)

def calculate_atr(high, low, close, period=14):
    """
    Calculate Average True Range (ATR)
    True Range = max(high-low, |high-close_prev|, |low-close_prev|)
    ATR = Average of True Range over period
    """
    high_low = high - low
    high_close_prev = np.abs(high - close.shift(1))
    low_close_prev = np.abs(low - close.shift(1))
    
    true_range = np.maximum(high_low, np.maximum(high_close_prev, low_close_prev))
    atr = true_range.rolling(window=period).mean()
    
    return atr, true_range

# Calculate ATR
df['ATR_14'], df['True_Range'] = calculate_atr(df['high'], df['low'], df['close'])

# Additional volatility measures
df['Price_Range'] = df['high'] - df['low']  # Daily range
df['Price_Range_Pct'] = (df['Price_Range'] / df['close']) * 100  # Range as % of close
df['Close_to_Close_Volatility'] = df['close'].pct_change().rolling(window=20).std() * np.sqrt(252)  # Annualized

# Volatility-based indicators
df['ATR_ratio'] = df['ATR_14'] / df['close']  # ATR as % of price
df['Volatility_Breakout'] = (df['True_Range'] > df['ATR_14'] * 2).astype(int)  # High volatility periods

print("✅ ATR and volatility indicators calculated")

print(f"\n📊 Volatility Statistics:")
volatility_stats = df[['ATR_14', 'Price_Range_Pct', 'Close_to_Close_Volatility', 'ATR_ratio']].describe()
display(volatility_stats)

print(f"\n🔍 Volatility Analysis:")
high_vol_days = df['Volatility_Breakout'].sum()
avg_atr_ratio = df['ATR_ratio'].mean() * 100
print(f"   • High volatility breakout days: {high_vol_days}")
print(f"   • Average ATR as % of price: {avg_atr_ratio:.2f}%")
print(f"   • Average daily price range: {df['Price_Range_Pct'].mean():.2f}%")

print(f"\n📋 Volatility Indicators Preview:")
vol_columns = [col for col in df.columns if any(x in col for x in ['ATR', 'Range', 'Volatility'])]
display(df[vol_columns].tail(10))


VOLATILITY INDICATORS CALCULATION
✅ ATR and volatility indicators calculated

📊 Volatility Statistics:


,ATR_14,Price_Range_Pct,Close_to_Close_Volatility,ATR_ratio
count,985.000000,999.000000,979.000000,985.000000
mean,3.878437,3.237782,0.306962,0.032689
std,1.157672,1.489365,0.050066,0.003702
min,2.244665,0.080982,0.174464,0.024099
25%,3.179034,2.147225,0.268860,0.029855
50%,3.605397,3.090537,0.309408,0.032241
75%,4.136721,4.133408,0.343144,0.035103
max,8.314932,9.391564,0.432854,0.043841



🔍 Volatility Analysis:
   • High volatility breakout days: 19
   • Average ATR as % of price: 3.27%
   • Average daily price range: 3.24%

📋 Volatility Indicators Preview:


,ATR_14,True_Range,Price_Range,Price_Range_Pct,Close_to_Close_Volatility,ATR_ratio,Volatility_Breakout
2022-09-17,7.607098,6.709964,6.709964,3.270710,0.321090,0.037080,0
2022-09-18,7.273063,11.216257,11.216257,5.697049,0.344358,0.036942,0
2022-09-19,7.154987,6.059889,6.059889,3.091724,0.333340,0.036504,0
2022-09-20,7.156053,5.139823,5.139823,2.657226,0.333475,0.036996,0
2022-09-21,7.332530,9.878959,9.878959,5.209048,0.336389,0.038663,0
2022-09-22,6.839770,2.187580,2.187580,1.159421,0.317629,0.036251,0
2022-09-23,6.864625,11.326978,11.326978,5.792172,0.351272,0.035103,0
2022-09-24,7.062968,6.550767,6.550767,3.305783,0.314417,0.035643,0
2022-09-25,7.149764,5.885684,5.885684,3.002958,0.296069,0.036479,0
2022-09-26,6.653955,4.084632,4.084632,2.059423,0.293768,0.033548,0


In [10]:
# ============================================
# Additional Technical Indicators
# ============================================

print("=" * 60)
print("ADDITIONAL TECHNICAL INDICATORS")
print("=" * 60)

# Williams %R
def calculate_williams_r(high, low, close, period=14):
    """Williams %R: %R = (Highest High - Close) / (Highest High - Lowest Low) * -100"""
    highest_high = high.rolling(window=period).max()
    lowest_low = low.rolling(window=period).min()
    williams_r = ((highest_high - close) / (highest_high - lowest_low)) * -100
    return williams_r

df['Williams_R'] = calculate_williams_r(df['high'], df['low'], df['close'])

# Commodity Channel Index (CCI)
def calculate_cci(high, low, close, period=20):
    """CCI = (Typical Price - SMA of Typical Price) / (0.015 * Mean Deviation)"""
    typical_price = (high + low + close) / 3
    sma_tp = typical_price.rolling(window=period).mean()
    mean_deviation = typical_price.rolling(window=period).apply(lambda x: np.abs(x - x.mean()).mean())
    cci = (typical_price - sma_tp) / (0.015 * mean_deviation)
    return cci

df['CCI_20'] = calculate_cci(df['high'], df['low'], df['close'])

# Money Flow Index (MFI)
def calculate_mfi(high, low, close, volume, period=14):
    """Money Flow Index: Similar to RSI but volume-weighted"""
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    
    positive_flow = money_flow.where(typical_price > typical_price.shift(1), 0)
    negative_flow = money_flow.where(typical_price < typical_price.shift(1), 0)
    
    positive_mf = positive_flow.rolling(window=period).sum()
    negative_mf = negative_flow.rolling(window=period).sum()
    
    mfi = 100 - (100 / (1 + positive_mf / negative_mf))
    return mfi

df['MFI_14'] = calculate_mfi(df['high'], df['low'], df['close'], df['volume'])

# Rate of Change (ROC)
df['ROC_10'] = ((df['close'] - df['close'].shift(10)) / df['close'].shift(10)) * 100

# Ultimate Oscillator (simplified version)
def calculate_ultimate_oscillator(high, low, close, period1=7, period2=14, period3=28):
    """Simplified Ultimate Oscillator"""
    bp = close - np.minimum(low, close.shift(1))  # Buying pressure
    tr = np.maximum(high, close.shift(1)) - np.minimum(low, close.shift(1))  # True range
    
    avg1 = bp.rolling(period1).sum() / tr.rolling(period1).sum()
    avg2 = bp.rolling(period2).sum() / tr.rolling(period2).sum()
    avg3 = bp.rolling(period3).sum() / tr.rolling(period3).sum()
    
    uo = 100 * ((4 * avg1) + (2 * avg2) + avg3) / 7
    return uo

df['Ultimate_Oscillator'] = calculate_ultimate_oscillator(df['high'], df['low'], df['close'])

print("✅ Additional technical indicators calculated:")
print("   • Williams %R")
print("   • Commodity Channel Index (CCI)")
print("   • Money Flow Index (MFI)")
print("   • Rate of Change (ROC)")
print("   • Ultimate Oscillator")

print(f"\n📊 Additional Indicators Statistics:")
additional_indicators = ['Williams_R', 'CCI_20', 'MFI_14', 'ROC_10', 'Ultimate_Oscillator']
display(df[additional_indicators].describe())

print(f"\n📋 Additional Indicators Preview:")
display(df[additional_indicators].tail(10))


ADDITIONAL TECHNICAL INDICATORS
✅ Additional technical indicators calculated:
   • Williams %R
   • Commodity Channel Index (CCI)
   • Money Flow Index (MFI)
   • Rate of Change (ROC)
   • Ultimate Oscillator

📊 Additional Indicators Statistics:


,Williams_R,CCI_20,MFI_14,ROC_10,Ultimate_Oscillator
count,986.000000,980.000000,986.000000,989.000000,971.000000
mean,-49.488848,12.816996,52.136015,0.819406,51.092322
std,29.384726,110.943692,17.453091,6.201113,9.359477
min,-99.826831,-278.048426,11.735021,-15.635478,25.756008
25%,-74.959840,-76.188334,39.347889,-3.444173,44.693958
50%,-50.328469,8.895267,51.682806,0.310682,51.099737
75%,-22.302898,98.290229,64.019904,4.518991,57.351874
max,-0.005899,350.911470,100.000000,21.095183,78.578266



📋 Additional Indicators Preview:


,Williams_R,CCI_20,MFI_14,ROC_10,Ultimate_Oscillator
2022-09-17,-40.547019,-16.163927,63.104341,1.689955,51.168138
2022-09-18,-68.465014,-52.370014,61.948663,-3.283017,47.731118
2022-09-19,-73.264100,-93.651965,62.940594,-1.863947,51.411213
2022-09-20,-87.392064,-108.753699,59.604759,-3.109414,45.085771
2022-09-21,-83.422856,-159.857943,56.168689,-5.042556,41.107337
2022-09-22,-87.534470,-170.747353,49.326030,-3.337860,36.898624
2022-09-23,-56.049615,-116.269390,56.374716,-2.785804,46.995913
2022-09-24,-45.474324,-14.079057,63.946501,-3.237513,47.975406
2022-09-25,-54.264566,-45.344187,57.438792,-3.917933,54.618972
2022-09-26,-44.752085,-20.478328,65.111070,-2.870386,57.099809


In [14]:
# ============================================
# Technical Indicators Summary and Export - FIXED
# ============================================

print("=" * 60)
print("TECHNICAL INDICATORS SUMMARY")
print("=" * 60)

# Compile all technical indicator features
technical_features = []

# Moving Averages
ma_features = [col for col in df.columns if any(x in col for x in ['SMA_', 'EMA_', 'VWAP', 'ratio'])]
technical_features.extend(ma_features)

# Oscillators
oscillator_features = [col for col in df.columns if any(x in col for x in ['RSI_', 'Stoch_', 'Williams_R', 'CCI_', 'MFI_', 'Ultimate_'])]
technical_features.extend(oscillator_features)

# MACD Features
macd_features = [col for col in df.columns if 'MACD' in col]
technical_features.extend(macd_features)

# Bollinger Bands
bb_features = [col for col in df.columns if 'BB_' in col]
technical_features.extend(bb_features)

# Volatility Features
vol_features = [col for col in df.columns if any(x in col for x in ['ATR', 'Range', 'Volatility', 'True_Range'])]
technical_features.extend(vol_features)

# Additional Features
additional_features = ['ROC_10']
technical_features.extend(additional_features)

# Remove duplicates and sort
technical_features = sorted(list(set(technical_features)))

print(f"📊 TECHNICAL INDICATORS SUMMARY:")
print(f"   • Total technical features: {len(technical_features)}")
print(f"   • Moving averages: {len(ma_features)}")
print(f"   • Oscillators: {len(oscillator_features)}")
print(f"   • MACD features: {len(macd_features)}")
print(f"   • Bollinger Bands: {len(bb_features)}")
print(f"   • Volatility indicators: {len(vol_features)}")
print(f"   • Additional features: {len(additional_features)}")

# Create technical indicators dataset
tech_indicators_df = df[technical_features].copy()

# Remove rows with NaN values (due to rolling calculations)
tech_indicators_df = tech_indicators_df.dropna()

print(f"\n📈 Technical Indicators Dataset:")
print(f"   • Shape after removing NaN: {tech_indicators_df.shape}")
print(f"   • Date range: {tech_indicators_df.index.min().date()} to {tech_indicators_df.index.max().date()}")

# Display feature preview
print(f"\n📋 Technical Indicators Preview:")
display(tech_indicators_df.head())

# Export technical indicators - THIS IS SAFE
output_path = 'C:\\Users\\Faraz\\Documents\\StockPredictionPro\\notebooks\\outputs\\technical_indicators.csv'
tech_indicators_df.to_csv(output_path)
print(f"\n💾 Technical indicators saved to: {output_path}")

# Feature list for reference
feature_list_path = 'C:\\Users\\Faraz\\Documents\\StockPredictionPro\\notebooks\\outputs\\technical_indicators_list.txt'
with open(feature_list_path, 'w') as f:
    f.write("Technical Indicators Feature List\n")
    f.write("=" * 40 + "\n\n")
    for i, feature in enumerate(technical_features, 1):
        f.write(f"{i:2d}. {feature}\n")

print(f"📝 Feature list saved to: {feature_list_path}")

# Summary statistics
print(f"\n📊 Technical Indicators Summary Statistics:")
display(tech_indicators_df.describe())

print(f"\n✅ Technical indicators calculation completed successfully!")
print(f"   Ready for feature selection and model training")



TECHNICAL INDICATORS SUMMARY
📊 TECHNICAL INDICATORS SUMMARY:
   • Total technical features: 46
   • Moving averages: 12
   • Oscillators: 14
   • MACD features: 5
   • Bollinger Bands: 8
   • Volatility indicators: 7
   • Additional features: 1

📈 Technical Indicators Dataset:
   • Shape after removing NaN: (800, 46)
   • Date range: 2020-07-19 to 2022-09-26

📋 Technical Indicators Preview:


,ATR_14,ATR_ratio,BB_breakout_lower,BB_breakout_upper,BB_lower,BB_middle,BB_position,BB_squeeze,BB_upper,BB_width,CCI_20,Close_to_Close_Volatility,EMA_12,EMA_26,EMA_50,MACD,MACD_bearish_crossover,MACD_bullish_crossover,MACD_histogram,MACD_signal,MFI_14,Price_Range,Price_Range_Pct,ROC_10,RSI_14,RSI_21,RSI_overbought,RSI_oversold,SMA_10,SMA_20,SMA_200,SMA_5,SMA_50,SMA_ratio_20_50,SMA_ratio_5_20,Stoch_D,Stoch_K,Stoch_bearish_crossover,Stoch_bullish_crossover,Stoch_overbought,Stoch_oversold,True_Range,Ultimate_Oscillator,VWAP,Volatility_Breakout,Williams_R
2020-07-19,2.926241,0.032351,0,0,89.057955,93.631217,0.152641,1,98.204478,9.146523,-103.026571,0.228974,91.992386,91.937699,90.312639,0.054689,0,0,-0.750014,0.804703,45.400758,2.429464,2.685853,-1.727532,45.336951,49.057494,0,0,91.976532,93.631217,87.489987,91.009811,89.627451,1.044671,0.972003,13.424149,12.572700,0,0,0,1,2.429464,50.696980,93.996718,0,-87.427300
2020-07-20,2.973942,0.032497,0,0,88.919456,93.384790,0.290501,1,97.850125,8.930668,-86.753518,0.230226,91.918762,91.906301,90.359745,0.012462,0,0,-0.633792,0.646254,44.708722,1.812649,1.980738,-2.298482,48.880757,51.173281,0,0,91.761241,93.384790,87.448689,91.044406,89.813841,1.039759,0.974938,13.889597,24.911800,0,1,0,0,1.812649,49.562515,93.831541,0,-75.088200
2020-07-21,2.953191,0.031571,0,0,88.901220,93.340634,0.522664,1,97.780049,8.878829,-10.613404,0.235291,92.168470,92.027454,90.484534,0.141017,0,0,-0.404190,0.545207,50.385127,2.848906,3.045595,-0.608593,54.905822,54.934317,0,0,91.703963,93.340634,87.410819,91.419310,90.062329,1.036400,0.979416,33.398995,62.712484,0,0,0,0,2.848906,50.299940,93.803467,0,-37.287516
2020-07-22,2.921271,0.030570,0,0,88.892266,93.349473,0.747987,1,97.806681,8.914415,64.165592,0.245344,92.690264,92.289134,90.683577,0.401131,0,0,-0.115261,0.516392,56.942543,4.199536,4.394652,4.076894,59.963171,58.291428,0,0,92.078291,93.349473,87.367388,92.167356,90.342834,1.033280,0.987337,56.990786,83.348073,0,0,1,0,4.199536,56.468677,93.798594,0,-16.651927
2020-07-23,3.043796,0.032738,0,0,88.857533,93.247981,0.468901,1,97.638429,8.780897,25.308985,0.263562,92.734055,92.339932,90.773433,0.394124,0,0,-0.097814,0.491938,57.641688,4.408181,4.741259,0.861485,51.929375,52.982997,0,0,92.157703,93.247981,87.313211,92.808964,90.561262,1.029667,0.995292,64.858408,48.514667,1,0,0,0,4.408181,49.097380,93.708892,0,-51.485333



💾 Technical indicators saved to: C:\Users\Faraz\Documents\StockPredictionPro\notebooks\outputs\technical_indicators.csv
📝 Feature list saved to: C:\Users\Faraz\Documents\StockPredictionPro\notebooks\outputs\technical_indicators_list.txt

📊 Technical Indicators Summary Statistics:


,ATR_14,ATR_ratio,BB_breakout_lower,BB_breakout_upper,BB_lower,BB_middle,BB_position,BB_squeeze,BB_upper,BB_width,CCI_20,Close_to_Close_Volatility,EMA_12,EMA_26,EMA_50,MACD,MACD_bearish_crossover,MACD_bullish_crossover,MACD_histogram,MACD_signal,MFI_14,Price_Range,Price_Range_Pct,ROC_10,RSI_14,RSI_21,RSI_overbought,RSI_oversold,SMA_10,SMA_20,SMA_200,SMA_5,SMA_50,SMA_ratio_20_50,SMA_ratio_5_20,Stoch_D,Stoch_K,Stoch_bearish_crossover,Stoch_bullish_crossover,Stoch_overbought,Stoch_oversold,True_Range,Ultimate_Oscillator,VWAP,Volatility_Breakout,Williams_R
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,4.130640,0.032788,0.031250,0.086250,116.203646,125.054980,0.549674,0.535000,133.906315,17.702669,17.384349,0.308991,125.571937,124.596265,122.867277,0.975672,0.037500,0.038750,-0.023413,0.999085,52.394899,4.113322,3.256743,1.143991,53.239750,53.326481,0.113750,0.021250,125.710645,125.054980,113.673879,126.027354,122.849619,1.015715,1.007267,51.987537,52.039348,0.220000,0.221250,0.246250,0.181250,4.161140,51.421939,125.164179,0.020000,-47.960652
std,1.129551,0.003796,0.174101,0.280908,28.398417,31.383856,0.324319,0.499086,34.861344,10.187299,111.583523,0.052746,31.749670,30.558329,28.233875,3.187319,0.190102,0.193119,0.918507,2.999033,17.465382,2.192156,1.485553,6.360955,12.411087,10.292135,0.317706,0.144307,32.027440,31.383856,15.821861,32.291649,28.768031,0.048965,0.039171,27.486977,29.355261,0.414505,0.415348,0.431095,0.385466,2.172543,9.570010,31.398554,0.140088,29.355261
min,2.530697,0.024099,0.000000,0.000000,82.887129,92.464514,-0.213144,0.000000,95.219785,4.879470,-278.048426,0.174464,91.918762,91.906301,90.312639,-5.563273,0.000000,0.000000,-3.994602,-5.044221,14.242722,0.409221,0.383671,-15.635478,22.219003,27.916895,0.000000,0.000000,91.703963,92.464514,86.971122,90.958838,89.627451,0.898409,0.919291,0.549826,0.173169,0.000000,0.000000,0.000000,0.000000,0.409221,25.756008,92.401549,0.000000,-99.826831
25%,3.350853,0.029869,0.000000,0.000000,98.570473,104.813129,0.287741,0.000000,111.138996,10.891127,-71.974285,0.267205,105.041744,104.339813,104.265792,-1.108773,0.000000,0.000000,-0.516059,-0.914637,39.260428,2.566044,2.152669,-3.209973,44.457774,45.990835,0.000000,0.000000,105.015198,104.813129,106.470120,104.733081,103.949244,0.980760,0.981138,28.291432,27.697687,0.000000,0.000000,0.000000,0.000000,2.591670,44.824740,104.750109,0.000000,-72.302313
50%,3.761228,0.032455,0.000000,0.000000,107.851781,114.825127,0.552338,1.000000,123.969540,15.064223,19.350611,0.312340,115.073591,114.632646,112.908650,0.523393,0.000000,0.000000,-0.038397,0.615985,51.830805,3.714054,3.108047,0.880606,52.159672,52.656602,0.000000,0.000000,114.786736,114.825127,111.330477,115.145926,113.726439,1.013122,1.004711,51.976461,51.847923,0.000000,0.000000,0.000000,0.000000,3.782521,51.438720,115.071429,0.000000,-48.152077
75%,4.423048,0.035248,0.000000,0.000000,120.759407,128.684118,0.810016,1.000000,136.085625,21.734591,104.743225,0.347473,128.229722,127.996170,127.792556,2.437703,0.000000,0.000000,0.499475,2.420401,63.913908,5.318938,4.173925,4.879498,60.119969,58.923865,0.000000,0.000000,128.575890,128.684118,121.815328,129.325024,127.530954,1.049326,1.031057,76.891863,79.310253,0.000000,0.000000,0.000000,0.000000,5.355384,57.776691,128.863499,0.000000,-20.689747
max,8.314932,0.043841,1.000000,1.000000,202.809468,219.354824,1.380956,1.000000,240.256609,56.812129,350.911470,0.432854,222.454005,216.131144,208.388550,11.714483,1.000000,1.000000,2.763446,10.797574,100.000000,15.892739,9.39156


✅ Technical indicators calculation completed successfully!
   Ready for feature selection and model training


## Technical Indicators Summary

### 🎯 Indicators Calculated

**Moving Averages & Trends:**
- Simple Moving Averages (5, 10, 20, 50, 200 periods)
- Exponential Moving Averages (12, 26, 50 periods)
- Volume Weighted Average Price (VWAP)
- Moving average ratios for trend strength

**Momentum Oscillators:**
- Relative Strength Index (RSI) - 14 & 21 periods
- Stochastic Oscillator (%K, %D)
- Williams %R
- Money Flow Index (MFI)
- Ultimate Oscillator

**Trend Following:**
- MACD (Moving Average Convergence Divergence)
- MACD Signal Line and Histogram
- Bollinger Bands (Upper, Middle, Lower)
- Commodity Channel Index (CCI)

**Volatility Measures:**
- Average True Range (ATR)
- Price Range and Range Percentage
- Bollinger Band Width
- Volatility Breakout Detection

**Additional Indicators:**
- Rate of Change (ROC)
- Trading signals and crossovers
- Overbought/Oversold conditions

### 🚀 Integration with StockPredictionPro

**For Model Training:**
- All indicators saved as engineered features
- Ready for feature selection process
- Compatible with time series modeling

**For Trading Strategies:**
- Signal generation for buy/sell decisions
- Overbought/Oversold detection
- Trend strength measurement
- Volatility-based position sizing

### 📊 Next Steps

1. **Feature Selection**: Use these indicators in feature selection notebook
2. **Model Training**: Integrate with ML models for prediction
3. **Backtesting**: Test trading strategies using these signals
4. **Real-time Implementation**: Deploy for live trading decisions
